# SNSDK Wrapper usage

In [5]:
import snsdk
import os
import sys
import json

current_dir = os.getcwd()
utils_dir = os.path.abspath(os.path.join(current_dir, ".."))
repo_dir = os.path.abspath(os.path.join(utils_dir, ".."))

sys.path.append(utils_dir)
sys.path.append(repo_dir)

## init wrapper

In [ ]:
from src.sambastudio_util import SnsdkWrapper

In [ ]:
snsdk = SnsdkWrapper(config_path="./config.yaml")

projects

In [ ]:
snsdk.create_project()

2024-07-22 12:49:22,408 [INFO] Project with name 'example project' found with id 51b1fe13-dcdb-41e3-8a78-514da36937c8
2024-07-22 12:49:22,409 [INFO] Project with name 'example project' already exists with id '51b1fe13-dcdb-41e3-8a78-514da36937c8', using it


'51b1fe13-dcdb-41e3-8a78-514da36937c8'

In [ ]:
snsdk.list_projects()

[{'project_name': 'Benchmarking_Llama7b',
  'project_id': '8d752994-bfd1-4eee-98a8-9ca0d50b8b37',
  'status': 'Available',
  'user_id': 'amit.kushwaha'},
 {'project_name': 'Shared',
  'project_id': 'b9896d2e-5054-4937-978f-67413104d6ba',
  'status': 'Available',
  'user_id': 'varun.krishna'},
 {'project_name': 'SNSDK-E2E-Finetuning_Embeddings-Project',
  'project_id': 'e2d0c5dc-894e-461a-9730-a814904db9f7',
  'status': 'Available',
  'user_id': 'rodrigo.maldonado'},
 {'project_name': 'snsdk_test_project',
  'project_id': '78f670e3-6fce-4923-9e7f-390c8dc34be5',
  'status': 'Available',
  'user_id': 'jorge.piedrahita'},
 {'project_name': 'sql_finetuning',
  'project_id': '302411f2-a7f7-42b3-a056-13f99330d3f9',
  'status': 'Available',
  'user_id': 'jorge.piedrahita'},
 {'project_name': 'Test_Finetune_Embeddings',
  'project_id': '6f1d50bc-cd1b-4574-b901-70fe929f5aa4',
  'status': 'Available',
  'user_id': 'francesca.raimondi'}]

Jobs

In [ ]:
snsdk.run_job(project_name="example project", dataset_name="smol_sql_pretraining_mm")

2024-07-22 12:49:41,470 [INFO] Project with name 'example project' found with id 51b1fe13-dcdb-41e3-8a78-514da36937c8
2024-07-22 12:49:42,781 [INFO] Model 'Llama-2-7b-chat-hf' with id '6090d4ac-a7bd-4c46-b417-7f8e42cf7bdb' available for training and deployment found


{'models': [{'model_id': '47ee0428-f9f8-6d8c-d02e-1ac778b83eb9', 'model_checkpoint_name': 'CLIP ViT-B-32 Backbone (Deprecated)', 'app_id': '6c14325a-1be7-4e48-b38f-19b33745fc3b', 'user_id': '', 'tenant_id': '', 'checkpoint_type': 'pretrained', 'model_size_gb': 0.57, 'dataset': '', 'dataset_info': '', 'dataset_url': '', 'path': 'common/checkpoints/47ee0428-f9f8-6d8c-d02e-1ac778b83eb9/checkpoint', 'sambanova_provided': True, 'version': 1, 'job_config': '', 'description': 'CLIP is a multi-modal neural network trained on (image, text) pairs.\nUsing a ViT-B-32 Backbone for visual features and a language model for text features,\nCLIP projects both modalities to a shared latent space, with the dot product between them used as a similarity score.\nThis enables the network to predict relevant text snippets given an image, without directly optimizing for the task.\nIt achieves this through its zero-shot capabilities and has been shown to match the performance of ResNet50 on ImageNet.\nThis CLIP

2024-07-22 12:49:43,088 [INFO] Dataset with name 'smol_sql_pretraining_mm' found with id d80f6355-af9d-406b-95c5-c31854f36f2c
2024-07-22 12:49:43,382 [INFO] Job with name 'None' created: '{'job_id': '385632fa-284c-42a2-892a-b1bfb6161a7e', 'job_name': 'snsdk_test_job', 'job_type': 'train', 'user_id': 'jorge.piedrahita', 'project_id': '51b1fe13-dcdb-41e3-8a78-514da36937c8', 'tenant_id': '41ceaded-9f08-47ae-aa02-15f39c899618', 'rdu_arch': 'sn20', 'result_path': '', 'parallel_instances': 1, 'app_id': 'ec012370-6ffa-4a3a-b230-2c62613f1d89', 'model_checkpoint': '6090d4ac-a7bd-4c46-b417-7f8e42cf7bdb', 'checkpoint_id': '', 'dataset_id': 'd80f6355-af9d-406b-95c5-c31854f36f2c', 'description': 'snsdk test training project', 'status': 'CREATED', 'image_version': '', 'variant_set_version': '', 'variant_name': '', 'project_name': '', 'dataset_name': '', 'input_data_path': '', 'hyperparams': {'batch_size': '256', 'debug_mode': 'false', 'do_eval': 'false', 'dump_inputs': 'false', 'eval_steps': '50', '

'385632fa-284c-42a2-892a-b1bfb6161a7e'

In [ ]:
snsdk.check_job_progress(project_name="sql_finetuning")

2024-07-18 11:40:28,992 [INFO] Job with name 'snsdk_test_job' in project sql_finetuning found with id a876668b-fc48-49b9-af23-6b078c40ef84
2024-07-18 11:40:29,379 [ERROR] Failed to check job progress. Details: {'detail': 'view job action not authorized for the user', 'status_code': 403}


{'detail': 'view job action not authorized for the user', 'status_code': 403}


Exception: Error message: {'detail': 'view job action not authorized for the user', 'status_code': 403}

In [ ]:
snsdk.list_checkpoints(project_name="sql_finetuning")

2024-07-18 12:48:21,276 [INFO] Job with name 'snsdk_test_job' in project 'sql_finetuning' found with id 'a876668b-fc48-49b9-af23-6b078c40ef84'
2024-07-18 12:48:21,660 [ERROR] Failed to list checkpoints. Details: {'detail': 'view checkpoints action not authorized for the user', 'status_code': 403}


Exception: Error message: {'detail': 'view checkpoints action not authorized for the user', 'status_code': 403}

In [ ]:
snsdk.promote_checkpoint("a876668b-fc48-49b9-af23-6b078c40ef84-50", project_name="sql_finetuning")

2024-07-18 15:01:55,077 [INFO] Project with name 'sql_finetuning' found with id 302411f2-a7f7-42b3-a056-13f99330d3f9
2024-07-18 15:01:55,332 [INFO] Project with name 'sql_finetuning' found with id 302411f2-a7f7-42b3-a056-13f99330d3f9
2024-07-18 15:01:55,605 [INFO] Job with name 'snsdk_test_job' in project 'sql_finetuning' found with id 'a876668b-fc48-49b9-af23-6b078c40ef84'
2024-07-18 15:01:55,906 [ERROR] Failed to promote checkpoint 'a876668b-fc48-49b9-af23-6b078c40ef84-50' to model. Details: {'detail': 'checkpoint promote action is not authorized for the user', 'status_code': 403}


Exception: Error message: {'detail': 'checkpoint promote action is not authorized for the user', 'status_code': 403}

In [ ]:
snsdk.create_endpoint(project_name="sql_finetuning")

2024-07-19 09:00:07,470 [INFO] Project with name 'sql_finetuning' found with id 302411f2-a7f7-42b3-a056-13f99330d3f9
2024-07-19 09:00:07,767 [INFO] Model with name 'llama2_7b_fine_tuned_nstext2sql' found with id c828d8d6-bbc6-4315-8573-014a183ffa58
2024-07-19 09:00:07,953 [INFO] Endpoint with name 'test-endpoint-sql' not created it already exist with id 27c560eb-1857-49f4-b707-a5e9ab106b1b


'27c560eb-1857-49f4-b707-a5e9ab106b1b'

In [ ]:
snsdk.get_endpoint_details()

2024-07-19 09:20:54,363 [INFO] Project with name 'sql_finetuning' found with id 302411f2-a7f7-42b3-a056-13f99330d3f9


{'status': 'Live',
 'url': '/api/predict/generic/302411f2-a7f7-42b3-a056-13f99330d3f9/27c560eb-1857-49f4-b707-a5e9ab106b1b',
 'langchain wrapper env': {'SAMBASTUDIO_BASE_URL': 'https://sjc3-demo1.sambanova.net',
  'SAMBASTUDIO_BASE_URI': 'api/predict/generic',
  'SAMBASTUDIO_PROJECT_ID': '302411f2-a7f7-42b3-a056-13f99330d3f9',
  'SAMBASTUDIO_ENDPOINT_ID': '27c560eb-1857-49f4-b707-a5e9ab106b1b',
  'SAMBASTUDIO_API_KEY': '5c005055-de1e-4198-be8e-ae25ba9c6fb7'}}

In [ ]:
snsdk.delete_endpoint()

2024-07-19 09:24:40,647 [INFO] Project with name 'sql_finetuning' found with id 302411f2-a7f7-42b3-a056-13f99330d3f9
2024-07-19 09:24:41,195 [INFO] Endpoint 'test-endpoint-sql' deleted in project 'sql_finetuning'


## deployed endpoint usage

In [191]:
prompt="""[INST]<<SYS>>

Generate a query using valid SQLite to answer the following questions for the summarized tables schemas provided bellow.
Do not assume the values on the database tables before generating the SQL query, always generate a SQL that query what is asked. 
The query must be in the format: 
```sql
query
```
    
Example:

```sql
SELECT * FROM mainTable;
```
<</SYS>>
	
        
CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from Customer table:
CustomerId	FirstName	LastName	Company	Address	City	State	Country	PostalCode	Phone	Fax	Email	SupportRepId
1	Luís	Gonçalves	Embraer - Empresa Brasileira de Aeronáutica S.A.	Av. Brigadeiro Faria Lima, 2170	São José dos Campos	SP	Brazil	12227-000	+55 (12) 3923-5555	+55 (12) 3923-5566	luisg@embraer.com.br	3
2	Leonie	Köhler	None	Theodor-Heuss-Straße 34	Stuttgart	None	Germany	70174	+49 0711 2842222	None	leonekohler@surfeu.de	5
3	François	Tremblay	None	1498 rue Bélanger	Montréal	QC	Canada	H2G 1A7	+1 (514) 721-4711	None	ftremblay@gmail.com	3
*/


CREATE TABLE "Employee" (
	"EmployeeId" INTEGER NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"FirstName" NVARCHAR(20) NOT NULL, 
	"Title" NVARCHAR(30), 
	"ReportsTo" INTEGER, 
	"BirthDate" DATETIME, 
	"HireDate" DATETIME, 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60), 
	PRIMARY KEY ("EmployeeId"), 
	FOREIGN KEY("ReportsTo") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from Employee table:
EmployeeId	LastName	FirstName	Title	ReportsTo	BirthDate	HireDate	Address	City	State	Country	PostalCode	Phone	Fax	Email
1	Adams	Andrew	General Manager	None	1962-02-18 00:00:00	2002-08-14 00:00:00	11120 Jasper Ave NW	Edmonton	AB	Canada	T5K 2N1	+1 (780) 428-9482	+1 (780) 428-3457	andrew@chinookcorp.com
2	Edwards	Nancy	Sales Manager	1	1958-12-08 00:00:00	2002-05-01 00:00:00	825 8 Ave SW	Calgary	AB	Canada	T2P 2T3	+1 (403) 262-3443	+1 (403) 262-3322	nancy@chinookcorp.com
3	Peacock	Jane	Sales Support Agent	2	1973-08-29 00:00:00	2002-04-01 00:00:00	1111 6 Ave SW	Calgary	AB	Canada	T2P 5M5	+1 (403) 262-3443	+1 (403) 262-6712	jane@chinookcorp.com
*/


CREATE TABLE "Genre" (
	"GenreId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("GenreId")
)

/*
3 rows from Genre table:
GenreId	Name
1	Rock
2	Jazz
3	Metal
*/


CREATE TABLE "Invoice" (
	"InvoiceId" INTEGER NOT NULL, 
	"CustomerId" INTEGER NOT NULL, 
	"InvoiceDate" DATETIME NOT NULL, 
	"BillingAddress" NVARCHAR(70), 
	"BillingCity" NVARCHAR(40), 
	"BillingState" NVARCHAR(40), 
	"BillingCountry" NVARCHAR(40), 
	"BillingPostalCode" NVARCHAR(10), 
	"Total" NUMERIC(10, 2) NOT NULL, 
	PRIMARY KEY ("InvoiceId"), 
	FOREIGN KEY("CustomerId") REFERENCES "Customer" ("CustomerId")
)

/*
3 rows from Invoice table:
InvoiceId	CustomerId	InvoiceDate	BillingAddress	BillingCity	BillingState	BillingCountry	BillingPostalCode	Total
1	2	2021-01-01 00:00:00	Theodor-Heuss-Straße 34	Stuttgart	None	Germany	70174	1.98
2	4	2021-01-02 00:00:00	Ullevålsveien 14	Oslo	None	Norway	0171	3.96
3	8	2021-01-03 00:00:00	Grétrystraat 63	Brussels	None	Belgium	1000	5.94
*/


CREATE TABLE "InvoiceLine" (
	"InvoiceLineId" INTEGER NOT NULL, 
	"InvoiceId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	"UnitPrice" NUMERIC(10, 2) NOT NULL, 
	"Quantity" INTEGER NOT NULL, 
	PRIMARY KEY ("InvoiceLineId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("InvoiceId") REFERENCES "Invoice" ("InvoiceId")
)

/*
3 rows from InvoiceLine table:
InvoiceLineId	InvoiceId	TrackId	UnitPrice	Quantity
1	1	2	0.99	1
2	1	4	0.99	1
3	2	6	0.99	1
*/


CREATE TABLE "MediaType" (
	"MediaTypeId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("MediaTypeId")
)

/*
3 rows from MediaType table:
MediaTypeId	Name
1	MPEG audio file
2	Protected AAC audio file
3	Protected MPEG-4 video file
*/


CREATE TABLE "Playlist" (
	"PlaylistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("PlaylistId")
)

/*
3 rows from Playlist table:
PlaylistId	Name
1	Music
2	Movies
3	TV Shows
*/


CREATE TABLE "PlaylistTrack" (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES "Playlist" ("PlaylistId")
)

/*
3 rows from PlaylistTrack table:
PlaylistId	TrackId
1	3402
1	3389
1	3390
*/


CREATE TABLE "Track" (
	"TrackId" INTEGER NOT NULL, 
	"Name" NVARCHAR(200) NOT NULL, 
	"AlbumId" INTEGER, 
	"MediaTypeId" INTEGER NOT NULL, 
	"GenreId" INTEGER, 
	"Composer" NVARCHAR(220), 
	"Milliseconds" INTEGER NOT NULL, 
	"Bytes" INTEGER, 
	"UnitPrice" NUMERIC(10, 2) NOT NULL, 
	PRIMARY KEY ("TrackId"), 
	FOREIGN KEY("MediaTypeId") REFERENCES "MediaType" ("MediaTypeId"), 
	FOREIGN KEY("GenreId") REFERENCES "Genre" ("GenreId"), 
	FOREIGN KEY("AlbumId") REFERENCES "Album" ("AlbumId")
)

/*
3 rows from Track table:
TrackId	Name	AlbumId	MediaTypeId	GenreId	Composer	Milliseconds	Bytes	UnitPrice
1	For Those About To Rock (We Salute You)	1	1	1	Angus Young, Malcolm Young, Brian Johnson	343719	11170334	0.99
2	Balls to the Wall	2	2	1	U. Dirkschneider, W. Hoffmann, H. Frank, P. Baltes, S. Kaufmann, G. Hoffmann	342562	5510424	0.99
3	Fast As a Shark	3	2	1	F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman	230619	3990994	0.99
*/
        
    how many music genres are in the db?
    [/INST]"""

In [192]:
from langchain_community.llms import SambaStudio

llm = SambaStudio(
    sambastudio_base_url=endpoint_env.get("SAMBASTUDIO_BASE_URL"),
    sambastudio_base_uri=endpoint_env.get("SAMBASTUDIO_BASE_URI"),
    sambastudio_project_id=endpoint_env.get("SAMBASTUDIO_PROJECT_ID"),
    sambastudio_endpoint_id=endpoint_env.get("SAMBASTUDIO_ENDPOINT_ID"),
    sambastudio_api_key=endpoint_env.get("SAMBASTUDIO_API_KEY"),
    model_kwargs = {
        "do_sample": True, 
        "temperature": 0.01,
        "max_tokens_to_generate": 512
    }
)
llm.invoke(prompt)

' SELECT COUNT(*) FROM "Genre"'